In [1]:
import segmentation.unet

import numpy as np
from scipy import ndimage
from matplotlib import pyplot as plt
import imageio
import cv2
import torchvision.transforms as transforms
import torch
from PIL import Image
import torch.nn.functional as F
from torch.utils.data import DataLoader

from segmentation.unet import smp_UNet


In [17]:
# code here modified from https://github.com/milesial/Pytorch-UNet/tree/e36c782fbfc976b7326182a47dd7213bd3360a7e
import logging
from pathlib import Path
import imageio
import cv2
import os

import numpy as np
import torch
from torch.utils.data import Dataset
import torchvision.transforms as transforms

from segmentation.helper_functions.general_functions import extract_image_names_from_folder


class BasicDataset(Dataset):
    def __init__(self, images_dir: str, n_channels: int, scale: float = 1.0,
                 augmentations=None, padding: bool = False, image_names=None):
        """
        TODO: fill in documentation
        :param images_dir:
        :param masks_dir:
        :param n_channels:
        :param scale:
        :param mask_suffix:
        :param augmentations:
        :param padding:
        """

        assert (n_channels == 1 or n_channels == 3), 'Dataset number of channels must be either 1 or 3'
        assert 0 < scale <= 1, 'Image scaling must be between 0 and 1'

        self.images_dir = Path(images_dir)
        self.n_channels = n_channels
        self.scale = scale
        self.standard_image_transform = transforms.Compose([transforms.ToTensor()])
        if image_names is not None:
            self.image_names = image_names
        else:
            self.image_names = extract_image_names_from_folder(images_dir)

        self.augmentations = augmentations
        self.padding = padding

        if padding:
            max_dimension = 0
            # loops through provided images and extracts the largest image dimension, for use if padding is selected
            for root, dirs, files in os.walk(self.images_dir):
                for name in files:
                    if name == '.DS_Store':
                        continue
                    image_file = os.path.join(root, name)
                    image = imageio.imread(image_file)  # TODO: investigate the warning here...
                    max_dimension = max(max_dimension, image.shape[0], image.shape[1])
            max_dimension = 32 * (max_dimension // 32 + 1)  # to be divisible by 32 TODO: why?

            self.max_dimension = max_dimension

        if not self.image_names:
            raise RuntimeError(f'No images found in {images_dir}, make sure you put your images there')
        logging.info(f'Creating dataset with {len(self.image_names)} examples')

    def __len__(self):
        return len(self.image_names)

    def set_augmentations(self, augmentations):
        self.augmentations = augmentations

    @staticmethod
    def load_image(self, filename, n_channels):
        image = imageio.imread(filename)

        # Converts to desired number of channels
        if n_channels == 1:  # Target input: 1 channel
            if image.shape[-1] == 3:  # Actual input: 3 channels
                image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
            elif image.shape[-1] == 4:  # Actual input: 4 channels
                image = cv2.cvtColor(image, cv2.COLOR_RGBA2GRAY)
            # No change required for already grayscale images
        elif n_channels == 3:  # Target input: 3 channels
            if image.shape[-1] == 4:  # Actual input: 4 channels
                image = cv2.cvtColor(image, cv2.COLOR_RGBA2RGB)
            elif image.shape[-1] != 3:  # Actual input: 1 channels
                image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)

        # Normalizing image
        if image.dtype == 'uint8':
            max_val = 255
        elif image.dtype == 'uint16':
            max_val = 65535
        else:
            raise RuntimeError('Image type not recognized.')

        image = image.astype(np.float32) / max_val

        return image

    def __getitem__(self, idx):
        # print('Dataloader is %s, image IDX is: %s, image_name is %s' % ('validation' if not self.augmentations else 'Training', idx, self.image_names[idx]))
        # return np.zeros((5,5))
        img_file = self.image_names[idx]

        img_array = self.load_image(self, filename=img_file, n_channels=self.n_channels)

        if self.augmentations:
            sample = self.augmentations(image=img_array, mask=mask_array)
            img_array = sample['image']
            mask_array = sample['mask']

        if self.padding:
            top = (self.max_dimension - img_array.shape[0]) // 2
            bottom = self.max_dimension - img_array.shape[0] - top
            left = (self.max_dimension - img_array.shape[1]) // 2
            right = self.max_dimension - img_array.shape[1] - left

            img_array = np.pad(img_array, pad_width=((top, bottom), (left, right)), mode='constant')

        img_tensor = self.standard_image_transform(img_array)

        return img_tensor

In [19]:
images_path = "../../../scratch_data/q2"

checkpoint_file_path = "/Users/matt/Documents/PhD/research_output/Automatic_Gel_Analyzer/segmentation_models/smp-UNet_No_Augmentations/checkpoints/checkpoint_epoch390.pth"
n_channels = 1
net = smp_UNet(
            encoder_name="resnet18",  # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
            in_channels=1,  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
            classes=2,  # model output channels (number of classes in your dataset)
        )


In [21]:

test_set = BasicDataset(images_path, n_channels, padding=True)
n_test = int(len(test_set))
test_loader = DataLoader(test_set, shuffle=False, batch_size=1, num_workers=0, pin_memory=True)
net.eval()
saved_dict = torch.load(f=checkpoint_file_path, map_location=torch.device("cpu"))
net.load_state_dict(saved_dict['network'])
# net.load_state_dict(saved_dict)
print(f'Model loaded from {checkpoint_file_path}')

# fig, ax = plt.subplots(n_test, 3, figsize=(10, 120))

plot_row = 0
for image in test_loader:

    with torch.no_grad():
        mask_pred = net(image)
    image = image.squeeze()
    mask_pred.squeeze()

    mask_pred_array = np.transpose(mask_pred.detach().squeeze().cpu().numpy(), (1, 2, 0))  # CHW to HWC
    height, width = mask_pred_array.shape[0], mask_pred_array.shape[1]

    threshold = 0.8
    thresholded = np.zeros((height, width))
    for row in range(height):
        for column in range(width):
            if mask_pred_array[row][column][0] < (1-threshold) and mask_pred_array[row][column][1] > threshold:
                thresholded[row][column] = 1

    # use a boolean condition to find where pixel values are > 0.75
    blobs = thresholded == 1

    # label connected regions that satisfy this condition
    labels, nlabels = ndimage.label(blobs, structure=[[1,1,1],[1,1,1],[1,1,1]])


    # find their centres of mass. in this case I'm weighting by the pixel values in
    # `img`, but you could also pass the boolean values in `blobs` to compute the
    # unweighted centroids.
    r, c = np.vstack(ndimage.center_of_mass(thresholded, labels, np.arange(nlabels) + 1)).T

    # find their distances from the top-left corner
    d = np.sqrt(r*r + c*c)


    # Get coordinates for each unique band
    # create array of intensities
    volume_labels = np.zeros((nlabels+1), float)
    area_labels = np.zeros((nlabels+1), int)

    for h in range(height):
        for w in range(width):
            volume_labels[labels[h][w]] += image[h][w]  # index = label, value += intensity(between 0 and 1)
            area_labels[labels[h][w]] += 1
    # print(f'(nlabels = {nlabels})\nvolume_labels: {volume_labels}')

# plot

# ax[0].imshow(thresholded)
    original_image = image.detach().squeeze().cpu().numpy()

    plt.figure(figsize=(10, 10))
    plt.imshow(original_image, cmap='gray')
    plt.imshow(np.ma.masked_array(labels, ~blobs), cmap=plt.cm.rainbow)


#     ax[plot_row][0].imshow(original_image, cmap='gray')
#
#     ax[plot_row][1].imshow(original_image, cmap='gray')
#     ax[plot_row][1].imshow(np.ma.masked_array(labels, ~blobs), cmap=plt.cm.rainbow)
# #         for ri, ci, di in zip(r, c, d):
# #             label = int(labels[int(ri)][int(ci)])
# #             ax[plot_row][1].annotate(f'{label}', xy=(ci, ri),  xytext=(0, -5),
# #                        textcoords='offset points', ha='center', va='top',
# #                        fontsize=8, color='blue')
#
#     ax[plot_row][2].imshow(np.ma.masked_array(labels, ~blobs), cmap=plt.cm.rainbow)
#     for ri, ci, di, count in zip(r, c, d, range(nlabels)):
#     #     ax[1].annotate('', xy=(0, 0), xytext=(ci, ri),
#     #                    arrowprops={'arrowstyle':'<-', 'shrinkA':0})
#     #     ax[1].annotate(f'label={label}, concentration={volume_labels[label]}', xy=(ci, ri),  xytext=(0, -5),
#     #     textcoords='offset points', ha='center', va='top',
#         label = int(labels[int(ri)][int(ci)])
#         ax[plot_row][2].annotate(f'{count+1}: V={round(volume_labels[count+1], 1)}', xy=(ci, ri),  xytext=(0, -5),
#                               textcoords='offset points', ha='center', va='top',
#                               fontsize=8)
#     plot_row+=1
#
#     # TODO: delete
#     # print(volume_labels)
#     break
# for aa in ax.flat:
#     aa.set_axis_off()
# fig.tight_layout()


In [22]:
plt.figure(figsize=(10, 10))
plt.imshow(original_image, cmap='gray')
plt.imshow(np.ma.masked_array(labels, ~blobs), cmap=plt.cm.rainbow)


In [99]:
np.unique(labeled_bands)

In [98]:
one_hot = F.one_hot(mask_pred.argmax(dim=1), net.n_classes).permute(0, 3, 1, 2).float()
onn = one_hot.numpy().squeeze()

input_marker_array = onn.argmax(axis=0)

input_marker_array = ndi.binary_fill_holes(input_marker_array)
labeled_bands, _ = ndi.label(input_marker_array)
plt.imshow(labeled_bands)
# plt.imshow(labeled_bands)

In [108]:
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

import skimage
from skimage.filters import sobel
from scipy import ndimage as ndi
from skimage.color import label2rgb


option = 'unique_bands'
one_hot = F.one_hot(mask_pred.argmax(dim=1), net.n_classes).permute(0, 3, 1, 2).float()
onn = one_hot.numpy().squeeze()

if option == 'unique_bands':
    input_marker_array = ndi.binary_fill_holes(onn.argmax(axis=0))
    input_marker_array, _ = ndi.label(input_marker_array)
    input_marker_array[input_marker_array > 0] = input_marker_array[input_marker_array > 0] + 1
else:
    input_marker_array = onn.argmax(axis=0)*2

input_marker_array[original_image < 0.3] = 1


# get thresholded array, convert to a marker array, then add background markers, and leave 0s for unfilled areas


    # Use Sobel filter on original image to find elevation map
elevation_map = sobel(original_image)

# Apply the watershed algorithm itself, using the elevation map and markers
segmentation = skimage.segmentation.watershed(elevation_map, input_marker_array)
original_seg = np.copy(segmentation)

if option != 'unique_bands':
    # Fill holes and relabel bands, giving each a unique label
    segmentation = ndi.binary_fill_holes(segmentation - 1)
    labeled_bands, _ = ndi.label(segmentation)
else:
    labeled_bands = segmentation - 1

# Overlay labels on original image
image_label_overlay = label2rgb(labeled_bands, image=original_image)

plt.figure(figsize=(10,10))
plt.axis('off')
plt.imshow(image_label_overlay)

In [86]:
view_image = np.copy(original_image)

view_image[view_image < 0.15] = 0

plt.figure(figsize=(10,10))
plt.imshow(input_marker_array,cmap='gray')


In [71]:
onn [1,0:5,0:5]

In [36]:
fig, ax = plt.subplots(1, 3, figsize=(20, 15))
ax[0].imshow(original_image, cmap='gray')
ax[1].imshow(mask_pred_array[...,0], cmap=plt.cm.rainbow)
ax[2].imshow(mask_pred_array[...,1], cmap=plt.cm.rainbow)
# plt.imshow(np.ma.masked_array(labels, ~blobs), cmap=plt.cm.rainbow)

In [48]:
plt.figure(figsize=(10, 10))
threshold = 0.99
thresholded = np.zeros((height, width))
for row in range(height):
    for column in range(width):
        if mask_pred_array[row][column][0] < (1-threshold) and mask_pred_array[row][column][1] > threshold:
            thresholded[row][column] = 1

plt.imshow(original_image, cmap='gray')
plt.imshow(onn[0,0,...], cmap='jet',alpha=0.5)


In [46]:
import torch.nn.functional as F

one_hot = F.one_hot(mask_pred.argmax(dim=1), net.n_classes).permute(0, 3, 1, 2).float()
onn = one_hot.numpy()

In [65]:
tstmask = mask_pred[0,:,0:3,0:3]
tstmask

In [67]:
mask_pred[0,:,0:3,0:3].argmax(dim=0)

In [68]:
rez = F.one_hot(mask_pred[0,:,0:3,0:3].argmax(dim=0),net.n_classes)
rez

In [53]:
torch.arange(0, 5)

In [54]:
F.one_hot(torch.arange(0, 5) )

In [14]:
from torchinfo import summary
net2 = smp_UNet(
            encoder_name="resnet18",  # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
            encoder_weights="ssl",  # use `imagenet` pretreined weights for encoder initialization
            in_channels=1,  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
            classes=2,  # model output channels (number of classes in your dataset)
        )
images_path = "../../../scratch_data/q2"

In [18]:
test_set = BasicDataset(images_path, n_channels, padding=True)
n_test = int(len(test_set))
test_loader = DataLoader(test_set, shuffle=False, batch_size=1, num_workers=0, pin_memory=True)
net2.eval()
# net.load_state_dict(saved_dict)
# fig, ax = plt.subplots(n_test, 3, figsize=(10, 120))

plot_row = 0
for image in test_loader:

    with torch.no_grad():
        mask_pred = net2(image)
    image = image.squeeze()
    mask_pred.squeeze()

    mask_pred_array = np.transpose(mask_pred.detach().squeeze().cpu().numpy(), (1, 2, 0))  # CHW to HWC
    height, width = mask_pred_array.shape[0], mask_pred_array.shape[1]

    threshold = 0.8
    thresholded = np.zeros((height, width))
    for row in range(height):
        for column in range(width):
            if mask_pred_array[row][column][0] < (1-threshold) and mask_pred_array[row][column][1] > threshold:
                thresholded[row][column] = 1

    # use a boolean condition to find where pixel values are > 0.75
    blobs = thresholded == 1

    # label connected regions that satisfy this condition
    labels, nlabels = ndimage.label(blobs, structure=[[1,1,1],[1,1,1],[1,1,1]])


    # find their centres of mass. in this case I'm weighting by the pixel values in
    # `img`, but you could also pass the boolean values in `blobs` to compute the
    # unweighted centroids.
    r, c = np.vstack(ndimage.center_of_mass(thresholded, labels, np.arange(nlabels) + 1)).T

    # find their distances from the top-left corner
    d = np.sqrt(r*r + c*c)


    # Get coordinates for each unique band
    # create array of intensities
    volume_labels = np.zeros((nlabels+1), float)
    area_labels = np.zeros((nlabels+1), int)

    for h in range(height):
        for w in range(width):
            volume_labels[labels[h][w]] += image[h][w]  # index = label, value += intensity(between 0 and 1)
            area_labels[labels[h][w]] += 1

    original_image = image.detach().squeeze().cpu().numpy()

    plt.figure(figsize=(10, 10))
    plt.imshow(original_image, cmap='gray')
    plt.imshow(np.ma.masked_array(labels, ~blobs), cmap=plt.cm.rainbow)

